Note, still in the process of experimenting with parameters

Import packages

In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pds
import datetime as dt
import talos as ta
from talos.model.normalizers import lr_normalizer

In [34]:
config = tf.ConfigProto()
config.intra_op_parallelism_threads = 8
config.inter_op_parallelism_threads = 8
tf.session(config=config)

AttributeError: module 'tensorflow' has no attribute 'session'

Use prepared dataset

In [12]:
data = pds.read_pickle('dataset_model2v2_w_label.pkl')
labels = data['label']
dataset = data.drop(columns=['label'])
dataset.head()

,1,2,3,4,5,6,7,8,9,10,11,hr_sin,hr_cos,dir_sin,dir_cos,speed,ttjs,latitude,longitude,journey_number
0,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326317,0.924521,1
1,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326237,0.925287,1
2,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326156,0.925670,1
3,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326076,0.926437,1
4,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.54024,-0.841511,0.0,0.0,0.326116,0.926437,1


Select an arbitrary number of test journeys

In [13]:
# Roughly 20%
num_test_segments = 700 
msk = np.random.randint(1, dataset.journey_number.unique()[-1], num_test_segments)
mask = dataset['journey_number'].isin(msk)

In [14]:
train_input = dataset[~mask]
test_input = dataset[mask]

train_labels = labels[~mask]
test_labels = labels[mask]

tr_dat = dataset.drop(columns=['journey_number'])

In [15]:
# Journey numbers were needed up until this point
train_data = train_input.drop(columns=['journey_number'])
test_data = test_input.drop(columns=['journey_number'])

Prepare model for parameter optimization

In [37]:
keras.backend.clear_session()

In [ ]:
def ogt_model(tr_d, tr_l, val_d, val_l, params):

    model = keras.Sequential([
        keras.layers.Dense(params['layer1'], 
                           activation=params['activation'], 
                           input_shape=(tr_d.shape[1])),
        keras.layers.Dense(params['layer2']),
        keras.layers.Dropout(params['dropout1']),
        keras.layers.Dense(params['layer3']),
        keras.layers.Dropout(params['dropout2']),
        keras.layers.Dense(params['layer4']),
        keras.layers.Dense(1, activation=params['last_activation'])
    ])
    
    model.compile(loss='mae', 
                  optimizer=params['optimizer'](lr_normalizer(params['lr'], params['optimizer'])),
                  metrics=['mae'])

    # Initial_epoch can be set to resume training after preciously run epochs
    out = model.fit(tr_d.values, 
                    tr_l.values, 
                    epochs = params['epochs'],
                    batch_size=params['batch_size'],
                    validation_data=[val_d, val_l])
   
    return out, model

Define parameters to optimize over

In [52]:
p = {'lr': (2, 10, 30),
     'layer1':[19, 32, 64],
     'layer2':[19, 32, 64],
     'layer3':[19, 32, 64],
     'layer4':[19, 32, 64],
     'batch_size': [2, 5, 10],
     'epochs': [10],
     'dropout1': (0, 0.40, 10),
     'dropout2': (0, 0.40, 10),
     'optimizer': [keras.optimizers.Adam, keras.optimizers.Nadam, keras.optimizers.Adadelta],
     #'losses': [categorical_crossentropy, logcosh],
     'activation':[tf.nn.sigmoid, tf.nn.relu, tf.nn.elu],
     'last_activation': [tf.nn.softmax, tf.nn.relu]}

pt = {'lr': [2, 0.04],
     'layer1':[1],
     'layer2':[1],
     'batch_size': [10, 30],
     'epochs': [1],
     'optimizer': [keras.optimizers.Adadelta],
     #'losses': [categorical_crossentropy, logcosh],
     'activation':[tf.nn.softmax],
     'last_activation': [tf.nn.relu]}

Run optimization

In [55]:
def simple_model(tr_d, tr_l, val_d, val_l, params):
    model = keras.Sequential([
        keras.layers.Dense(params['layer1'], 
                           activation=params['activation'], 
                           input_shape=(tr_d.shape[1],)),
        keras.layers.Dense(params['layer2']),
        keras.layers.Dense(1, activation=params['last_activation'])
    ])

    model.compile(loss='poisson', 
                  optimizer=params['optimizer'](lr_normalizer(params['lr'], params['optimizer'])),
                  metrics=['mae', 'accuracy'])

    out = model.fit(tr_d,
                    tr_l,
                    epochs = params['epochs'],
                    batch_size=params['batch_size'],
                    validation_data=[val_d, val_l])

 
    return out, model

h = ta.Scan(train_data.values, train_labels.values,
            params=pt,
            dataset_name='talos_testing',
            experiment_no='1',
            val_split=0,
            x_val = test_data.values,
            y_val = test_labels.values,
            model=simple_model,
            reduction_metric=keras.metrics.mae,
            clear_tf_session=False
           )











  0%|          | 0/4 [00:00<?, ?it/s]

Train on 2745777 samples, validate on 566223 samples
Epoch 1/1
1360230/2745777 [=============>................] - ETA: 2:52 - loss: 930.5746 - mean_absolute_error: 57.7348 - acc: 0.0121

KeyboardInterrupt: 

In [43]:
r = ta.Reporting(h)

In [49]:
r.high('mean_absolute_error')

'56.770474072168945'

In [50]:
r.correlate('mean_absolute_error')


ValueError: cannot insert mean_absolute_error, already exists